## Exercises

Do your work for this exercise in a jupyter notebook or python script named `tidy_data`. Save this work in your `classification-exercises` repo. Add, commit, and push your changes.

1. Attendance Data

    Read the data from the `attendance` table and calculate an attendance percentage for each student. One half day is worth 50% of a full day, and 10 tardies is equal to one absence.
    
    You should end up with something like this:

        name
        Billy    0.5250
        Jane     0.6875
        John     0.9125
        Sally    0.7625
        Name: grade, dtype: float64


In [97]:
import pandas as pd
import numpy as np
from env import get_connection
import seaborn as sns
import matplotlib.pyplot as plt
from pydataset import data
import os
from scipy import stats

#turn off the pink warning boxes
import warnings
warnings.filterwarnings('ignore')

from sklearn.model_selection import train_test_split
from sklearn.impute import SimpleImputer

import acquire
import prepare

from env import get_connection
pd.options.display.max_columns = None

%matplotlib inline

In [76]:
def get_attendance_data():
    if os.path.isfile('attendance.csv'):
        return pd.read_csv('attendance.csv')
    else:
        url = get_connection('tidy_data')
        query = '''
                SELECT *
                FROM attendance
                '''
        attendance = pd.read_sql(query, url)
        attendance.to_csv('attendance.csv')
        attendance.drop(columns='Unnamed: 0.1')
        return attendance

In [77]:
attendance = get_attendance_data()

In [78]:
attendance.head()

,Unnamed: 0.1,Unnamed: 0,2018-01-01,2018-01-02,2018-01-03,2018-01-04,2018-01-05,2018-01-06,2018-01-07,2018-01-08
0,0,Sally,P,T,T,H,P,A,T,T
1,1,Jane,A,P,T,T,T,T,A,T
2,2,Billy,A,T,A,A,H,T,P,T
3,3,John,P,T,H,P,P,T,P,P


In [64]:
# rename columns in the way we want it represented as categories in the day column. 
tidy_data = tidy_data.drop(columns='Unnamed: 0.1')

In [65]:
tidy_data.columns = ['name', '01-01', '01-02', '01-03', '01-04', '01-05', '01-06', '01-07', '01-08']

In [66]:
tidy_data.head()

,name,01-01,01-02,01-03,01-04,01-05,01-06,01-07,01-08
0,Sally,P,T,T,H,P,A,T,T
1,Jane,A,P,T,T,T,T,A,T
2,Billy,A,T,A,A,H,T,P,T
3,John,P,T,H,P,P,T,P,P


In [67]:
# melt the data into 3 columns: product, year, and sales
tidy_data = tidy_data.melt(id_vars=['name'],
                         var_name='days',
                         value_name='attendance')

In [68]:
tidy_data['attendance'] = tidy_data['attendance'].map({'A': 0, 'H':.5, 'T':.9, 'P':1})

In [69]:
tidy_data.head()

,name,days,attendance
0,Sally,01-01,1.0
1,Jane,01-01,0.0
2,Billy,01-01,0.0
3,John,01-01,1.0
4,Sally,01-02,0.9


In [70]:
tidy_data.groupby('name')['attendance'].mean()

name
Billy    0.5250
Jane     0.6875
John     0.9125
Sally    0.7625
Name: attendance, dtype: float64

2. Coffee Levels

    1. Read the `coffee_levels` table.
    1. Transform the data so that each carafe is in it's own column.
    1. Is this the best shape for the data?

In [98]:
def get_coffee_data():
    if os.path.isfile('coffee_levels.csv'):
        return pd.read_csv('coffee_levels.csv')
    else:
        url = get_connection('tidy_data')
        query = '''
                SELECT *
                FROM coffee_levels;
                '''
        coffee_levels = pd.read_sql(query, url)
        coffee_levels.to_csv('coffee_levels.csv')
        coffee_levels.drop(columns='Unnamed: 0.0')
        return coffee_levels

In [99]:
coffee = get_coffee_data()

In [100]:
coffee.head()

,Unnamed: 0,hour,coffee_carafe,coffee_amount
0,0,8,x,0.816164
1,1,9,x,0.451018
2,2,10,x,0.843279
3,3,11,x,0.335533
4,4,12,x,0.898291


3. Cake Recipes

    1. Read the `cake_recipes` table. This data set contains cake tastiness scores for combinations of different recipes, oven rack positions, and oven temperatures.
    1. Tidy the data as necessary.
    1. Which recipe, on average, is the best? recipe b
    1. Which oven temperature, on average, produces the best results? 275
    1. Which combination of recipe, rack position, and temperature gives the best result? recipe b, bottom rack, 300 degrees